In [ ]:
import os
import spacy
import warnings
import wordcloud
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import numpy as np
import pandas as pd 
import os
import re
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import textstat
import spacy
nlp = spacy.load('en_core_web_sm')

from spacy import displacy
from nltk.tokenize import sent_tokenize, word_tokenize 

import warnings
warnings.filterwarnings("ignore")

import contractions
import nltk
import string

#nltk.download('stopwords')
#from nltk.corpus import stopwords
#stop_words = set(stopwords.words('english'))

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')
nltk.download('omw-1.4')
import pkg_resources
from symspellpy import SymSpell, Verbosity

In [ ]:
patient_notes = pd.read_csv("patient_notes.csv")
#one case number has many pn_num
features = pd.read_csv("features.csv")
#one case has many feature_num
train = pd.read_csv("train.csv")

train = train.merge(features, on=['feature_num', 'case_num'], how='left')
train = train.merge(patient_notes, on=['pn_num', 'case_num'], how='left')

pd.set_option('display.max_colwidth', None)
display(train.head())

### Pre-processing

In [ ]:
train["pn_history_lower"] = train["pn_history"].str.lower()
regex = re.compile(r'<[^>]+>')
train["pn_history_noContractions"] = train["pn_history_lower"].apply(contractions.fix)

def remove_unicode_chars(text):
    text = text.encode("ascii", "ignore").decode()
    return text
train["pn_history_noUnicode"] = train["pn_history_noContractions"].apply(remove_unicode_chars)

def remove_punctuations(text):
    text = re.sub('[%s]' % re.escape(string.punctuation), " ",text)
    return text
train["pn_history_noPuncts"] = train["pn_history_noUnicode"].apply(remove_punctuations)

#def remove_stopwords(text):
#    return " ".join([word for word in str(text).split() if word not in stop_words])
#train["tweet_noStopwords"] = train["tweet_noPuncts"].apply(remove_stopwords)
def remove_extra_spaces(text):
    text = re.sub(' +', ' ', text).strip()
    return text
train["pn_history_noES"] = train["pn_history_noPuncts"].apply(remove_extra_spaces)
def lemmatize_text(text):
    words = [lemmatizer.lemmatize(word) for word in text.split()]
    text = ' '.join(words)
    return text
train["pn_history_preprocessed"] = train["pn_history_noES"].apply(lemmatize_text)

In [ ]:
train.shape

In [ ]:
train = train.drop_duplicates(["pn_history_preprocessed"])

In [ ]:
train.shape

## Model 1 : using KeyBERT with all-distilroberta-v1

In [ ]:
from keybert import KeyBERT
from tqdm import tqdm

In [ ]:
model = KeyBERT(model="all-distilroberta-v1")

### Extracting 2 keywords 

In [ ]:
l = []
for index, row in tqdm(train.iterrows()):
    l.append(model.extract_keywords(
                row['pn_history_preprocessed'],
                top_n=3, 
                keyphrase_ngram_range=(1, 2),
                use_mmr=True, 
                diversity=0.7))

In [ ]:
train = train.reset_index(drop=True)

In [ ]:
train = train.join(pd.DataFrame(l, columns=['first_keyword', 'second_keyword', 'third_keyword']))

In [ ]:
#saving the file
train.to_csv("two_keywords_extraction.csv")

In [ ]:
#seprating keyword along with cosine similarity
def func(x):
    return x[1]
train['first_keyword_score'] = train.first_keyword.apply(lambda x: func(x))
def func(x):
    return x[0]
train['first_keyword'] = train.first_keyword.apply(lambda x: func(x))

def func(x):
    return x[1]
train['second_keyword_score'] = train.second_keyword.apply(lambda x: func(x))
def func(x):
    return x[0]
train['second_keyword'] = train.second_keyword.apply(lambda x: func(x))

def func(x):
    return x[1]
train['third_keyword_score'] = train.third_keyword.apply(lambda x: func(x))
def func(x):
    return x[0]
train['third_keyword'] = train.third_keyword.apply(lambda x: func(x))

In [ ]:
train.to_csv("two_keywords_extraction_with_score.csv")

### Extracting 3 keywords

In [ ]:

l = []
for index, row in tqdm(train.iterrows()):
    l.append(model.extract_keywords(
                row['pn_history_preprocessed'],
                top_n=3,
                keyphrase_ngram_range=(1, 3),
                use_mmr=True, 
                diversity=0.7))

In [ ]:
train_new = train.join(pd.DataFrame(l, columns=['first_keyword_three', 'second_keyword_three', 'third_keyword_three']))

In [ ]:
train.to_csv("three_keywords_extraction.csv")

In [ ]:
def func(x):
    return x[1]
train_new['first_keyword_score_three'] = train_new.first_keyword_three.apply(lambda x: func(x))
def func(x):
    return x[0]
train_new['first_keyword_three'] = train_new.first_keyword_three.apply(lambda x: func(x))

def func(x):
    return x[1]
train_new['second_keyword_score_three'] = train_new.second_keyword_three.apply(lambda x: func(x))
def func(x):
    return x[0]
train_new['second_keyword_three'] = train_new.second_keyword_three.apply(lambda x: func(x))

def func(x):
    return x[1]
train_new['third_keyword_score_three'] = train_new.third_keyword_three.apply(lambda x: func(x))
def func(x):
    return x[0]
train_new['third_keyword_three'] = train_new.third_keyword_three.apply(lambda x: func(x))

In [ ]:
train_new.to_csv("three_keywords_extraction_with_score.csv")

### Extracting 4 keywords

In [ ]:

l = []
for index, row in tqdm(train.iterrows()):
    l.append(model.extract_keywords(
                row['pn_history_preprocessed'],
                top_n=3,
                keyphrase_ngram_range=(1, 4),
                use_mmr=True, 
                diversity=0.7))

In [ ]:
train_new['third_keyword_score_three'] = train_new['third_keyword_score']

In [ ]:
train_new_new = train_new.join(pd.DataFrame(l, columns=['first_keyword_four', 'second_keyword_four', 'third_keyword_four']))


In [ ]:
train_new_new.to_csv("four_keywords_extraction.csv")

In [ ]:
def func(x):
    return x[1]
train_new_new['first_keyword_score_four'] = train_new_new.first_keyword_four.apply(lambda x: func(x))
def func(x):
    return x[0]
train_new_new['first_keyword_four'] = train_new_new.first_keyword_four.apply(lambda x: func(x))

def func(x):
    return x[1]
train_new_new['second_keyword_score_four'] = train_new_new.second_keyword_four.apply(lambda x: func(x))
def func(x):
    return x[0]
train_new_new['second_keyword_four'] = train_new_new.second_keyword_four.apply(lambda x: func(x))

def func(x):
    return x[1]
train_new_new['third_keyword_score_four'] = train_new_new.third_keyword_four.apply(lambda x: func(x))
def func(x):
    return x[0]
train_new_new['third_keyword_four'] = train_new_new.third_keyword_four.apply(lambda x: func(x))

In [ ]:
train_new_new.to_csv("all_keywords_with_score.csv")

In [ ]:
#saving a copy
train_with_scores = train_new_new.copy()

In [ ]:
train_with_scores = pd.read_csv("all_keywords_with_score.csv")

In [ ]:
train_with_scores = train_with_scores[['id', 'case_num', 'pn_num', 'feature_num', 'annotation', 'location', 'feature_text', 'pn_history', 'pn_history_lower', 'pn_history_noContractions', 'pn_history_noUnicode', 'pn_history_noPuncts', 'pn_history_noES', 'pn_history_preprocessed', 'first_keyword', 'second_keyword', 'third_keyword', 'first_keyword_score', 'second_keyword_score', 'third_keyword_score', 'first_keyword_three', 'second_keyword_three', 'third_keyword_three', 'first_keyword_score_three', 'second_keyword_score_three', 'third_keyword_score_three', 'first_keyword_four', 'second_keyword_four', 'third_keyword_four', 'first_keyword_score_four', 'second_keyword_score_four', 'third_keyword_score_four']]

In [ ]:
train_with_scores.head()

In [ ]:
train_with_scores = train_with_scores.iloc[:,14:]

### Analysing quantiles to choose a threshold value 

In [ ]:
train_with_scores.first_keyword_score.quantile([0.25, 0.5, 0.75, 0.9])

In [ ]:
train_with_scores.second_keyword_score.quantile([0.25, 0.5, 0.75, 0.9])

In [ ]:
train_with_scores.third_keyword_score.quantile([0.25, 0.5, 0.75, 0.9])

### selecting those keywords which have a cosine similarity more than 0.25

In [ ]:
train_with_scores.first_keyword = np.where(train_with_scores.first_keyword_score > 0.25, train_with_scores.first_keyword, np.nan)
train_with_scores.second_keyword = np.where(train_with_scores.second_keyword_score > 0.25, train_with_scores.second_keyword, np.nan)
train_with_scores.third_keyword = np.where(train_with_scores.third_keyword_score > 0.25, train_with_scores.third_keyword, np.nan)

In [ ]:
train_with_scores.first_keyword_score.median(), train_with_scores.second_keyword_score.median(), train_with_scores.third_keyword_score.median()


In [ ]:
train_with_scores.first_keyword_three = np.where(train_with_scores.first_keyword_score_three > 0.25, train_with_scores.first_keyword_three, np.nan)
train_with_scores.second_keyword_three = np.where(train_with_scores.second_keyword_score_three > 0.25, train_with_scores.second_keyword_three, np.nan)
train_with_scores.third_keyword_three = np.where(train_with_scores.third_keyword_score_three > 0.25, train_with_scores.third_keyword_three, np.nan)


In [ ]:
train_with_scores.first_keyword_four = np.where(train_with_scores.first_keyword_score_four > 0.25, train_with_scores.first_keyword_four, np.nan)
train_with_scores.second_keyword_four = np.where(train_with_scores.second_keyword_score_four > 0.25, train_with_scores.second_keyword_four, np.nan)
train_with_scores.third_keyword_four = np.where(train_with_scores.third_keyword_score_four > 0.25, train_with_scores.third_keyword_four, np.nan)


In [ ]:
train_with_scores = train_new_new[['id', 'pn_num', 'pn_history_preprocessed']].join(train_with_scores)

In [ ]:
train_with_scores[:1].third_keyword.values

### Predicting Annotations

In [ ]:
def func(x):
    all_annotation = []
    for annotation in [x.first_keyword, x.second_keyword, x.third_keyword, 
                       x.first_keyword_three, x.second_keyword_three, x.third_keyword_three,
                       x.first_keyword_four, x.second_keyword_four, x.third_keyword_four]:
        
        if str(annotation) != 'nan':
            all_annotation.append(str(annotation))
    return list(set(all_annotation))
    
train_with_scores['predicted_annotation'] = train_with_scores.apply(lambda x: func(x), axis=1)

In [ ]:
train_with_scores[['pn_num', 'predicted_annotation']]

In [ ]:
train_with_scores.to_csv("final_annotations")

## Evaluation Phase 1 : Using Fuzzy Wuzzy Techniques

In [ ]:
train_with_scores = pd.read_csv("final_annotations",index_col=[0])

In [ ]:
from thefuzz import fuzz
from thefuzz import process

In [ ]:
patient_notes_test = pd.read_csv("patient_notes.csv")
#one case number has many pn_num
features_test = pd.read_csv("features.csv")
#one case has many feature_num
test = pd.read_csv("train.csv")
test = test.merge(features_test, on=['feature_num', 'case_num'], how='left')
test = test.merge(patient_notes_test, on=['pn_num', 'case_num'], how='left')
test = test.merge(train_with_scores[['pn_num', 'pn_history_preprocessed']], on=['pn_num'], how='left')
pd.set_option('display.max_colwidth', None)
display(test.head(2))

In [ ]:
test = test.merge(train_with_scores[['pn_num', 'predicted_annotation']], how='left', on='pn_num')

In [ ]:
evaluation = test[['pn_num', 'annotation', 'predicted_annotation', 'pn_history', 'pn_history_preprocessed']]
evaluation = evaluation[evaluation.annotation != '[]']
evaluation['annotation'] = evaluation['annotation'].astype(str)
evaluation['ground_truth_annotation'] = evaluation.annotation.apply(lambda x: eval(x))
evaluation = evaluation.explode('ground_truth_annotation')
evaluation['ground_truth_annotation_junk'] = evaluation.ground_truth_annotation.apply(lambda x: len(str(x)) < 3)

In [ ]:
evaluation = evaluation.drop_duplicates()

In [ ]:
evaluation.ground_truth_annotation_junk.value_counts(normalize=True)

In [ ]:
evaluation_pp = evaluation.copy()

In [ ]:
evaluation.explode('predicted_annotation').shape

In [ ]:
evaluation['predicted_annotation'] = evaluation['predicted_annotation'].astype(str)
evaluation = evaluation[evaluation.predicted_annotation != '[]']
evaluation['prediction'] = evaluation.predicted_annotation.apply(lambda x: eval(x))
evaluation = evaluation.explode('prediction')

In [ ]:
#choosing the columns of interest
cols = ['pn_num', 
       'pn_history_preprocessed', 
       'ground_truth_annotation',
       'prediction']
evaluation = evaluation[cols]

In [ ]:
#getting the fuzzy wuzzy score
def get_fuzzy_wuzzy_score(x):
    prediction = x['prediction']
    label = x['ground_truth_annotation']
    return fuzz.partial_ratio(prediction, label)

evaluation['score'] = evaluation.apply(lambda x: get_fuzzy_wuzzy_score(x), axis=1)

In [ ]:
sns.distplot(evaluation.score)

### Evaluation Phase 2: Using Precision and Recall

#### PRECISION

In [ ]:
def relevance(x, threshold):
    all_predictions = x.groupby(['prediction']).agg({'score': 'max'}).reset_index()
    found_predictions = len(all_predictions[all_predictions['score'] >= threshold])
    return found_predictions, len(all_predictions)

In [ ]:
threshold_values = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
all_relevant_predictions = pd.DataFrame()

for threshold_val in threshold_values:
    relevant_predictions = evaluation.groupby(['pn_num']).apply(lambda x: relevance(x, threshold=threshold_val))
    relevant_predictions = relevant_predictions.reset_index()
    relevant_predictions = pd.DataFrame(relevant_predictions[0].tolist(), index=relevant_predictions.pn_num)\
                            .rename(columns={0:f'found_labels', 1:f'total_predictions'})\
                            .reset_index()
    relevant_predictions[f'precision'] = relevant_predictions[f'found_labels'] / relevant_predictions[f'total_predictions']
    relevant_predictions['threshold'] = threshold_val
    all_relevant_predictions = pd.concat([all_relevant_predictions, relevant_predictions], axis=0)

#### RECALL

In [ ]:
def retrieval(x, threshold):
    all_labels = x.groupby(['ground_truth_annotation']).agg({'score': 'max'}).reset_index()
    found_labels = len(all_labels[all_labels['score'] >= threshold])
    total_predictions = x['prediction'].nunique()
    
    return found_labels, len(all_labels)

In [ ]:
threshold_values = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
all_retrieved_predictions = pd.DataFrame()

for threshold_val in threshold_values:
    retrieved_labels = evaluation.groupby(['pn_num']).apply(lambda x: retrieval(x, threshold=threshold_val))
    retrieved_labels = retrieved_labels.reset_index()
    retrieved_labels = pd.DataFrame(retrieved_labels[0].tolist(), index=retrieved_labels.pn_num)\
                        .rename(columns={0:f'found_predictions', 1:f'total_labels'})\
                        .reset_index()
    retrieved_labels['recall'] = retrieved_labels[f'found_predictions'] / retrieved_labels[f'total_labels']
    retrieved_labels['threshold'] = threshold_val
    all_retrieved_predictions = pd.concat([all_retrieved_predictions, retrieved_labels], axis=0)

In [ ]:
evaluation_df = all_retrieved_predictions.merge(all_relevant_predictions, how='left', on=['pn_num', 'threshold'])
evaluation_df = evaluation_df[['pn_num', 'threshold', 'total_labels', 'total_predictions', 'found_labels', 'found_predictions', 'recall', 'precision']]

evaluation_df['f1_score'] = 2 * (evaluation_df['recall'] * evaluation_df['precision']) / (evaluation_df['recall'] + evaluation_df['precision'])
evaluation_df

### The below cell can be uncommented and run to check predictions of a particular patient number. This also has a filter for a ground truth annotation / label already present in the dataset and the predictions against it can be checked

In [ ]:
#Run this to check some particular evaluation or annotation
#evaluation[(evaluation.pn_num == 82843) & (evaluation.ground_truth_annotation=="increased appetite")]


#### Saving Precision, Recall and F1 scores to csv files

In [ ]:
#Joining it with the clinical text
all_distill_roberta_evaluation = evaluation_df.merge(test,on="pn_num",how="left")

In [ ]:
all_distill_roberta_evaluation.to_csv("all_distill_roberta_evaluation.csv", index=False)

In [ ]:
evaluation_df.to_csv("all_distill_roberta_evaluation_scores.csv",index=False)